# Problem set 3: Text analysis of DOJ press releases

**Total points (without extra credit)**: 52 

- For background:

    - DOJ is the federal law enforcement agency responsible for federal prosecutions; this contrasts with the local prosecutions in the Cook County dataset we analyzed earlier. Here's a short explainer on which crimes get prosecuted federally versus locally: https://www.criminaldefenselawyer.com/resources/criminal-defense/federal-crime/state-vs-federal-crimes.htm#:~:text=Federal%20criminal%20prosecutions%20are%20handled,of%20state%20and%20local%20law. 
    - Here's the Kaggle that contains the data: https://www.kaggle.com/jbencina/department-of-justice-20092018-press-releases 
    - Here's the code the dataset creator used to scrape those press releases here if you're interested: https://github.com/jbencina/dojreleases

## 0.0 Import packages

In [11]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
### nltk.download('averaged_perceptron_tagger')
### nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
### ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## lda
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

## 0.1 Load and clean text data

In [12]:
## first, unzip the file pset3_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

## 1. Tagging and sentiment scoring (17 points)

Focus on the following press release: `id` == "17-1204" about this pharmaceutical kickback prosecution: https://www.forbes.com/sites/michelatindera/2017/11/16/fentanyl-billionaire-john-kapoor-to-plead-not-guilty-in-opioid-kickback-case/?sh=21b8574d6c6c 

The `contents` column is the one we're treating as a document. You may need to to convert it from a pandas series to a single string.

We'll call the raw string of this press release `pharma`

In [13]:
pharma = doj[doj['id'] == "17-1204"]['contents'].iloc[0]
pharma

'The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain.\xa0"More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids. And yet some medical professionals would rather take advantage of the addicts than try to help them," said Attorney General Jeff Sessions. "This Justice Department will not tolerate this.\xa0 We will hold accountable anyone – from street dealers to corporate executives -- who illegally contributes to this nationwide epidemic.\xa0 And under the leadership of President Trump, we are fully committed to defeating this threat to the American people.”John N. Kapoor, 74, of Phoenix, Ariz., a current member of the Board of Directors of Insys, was arrested this morning in Arizona and charged with RICO conspi

### 1.1 part of speech tagging (3 points)

A. Preprocess the `pharma` press release to remove all punctuation / digits (you can use `.isalpha()` to subset)

B. With the preprocessed press release from part A, use the part of speech tagger within nltk to tag all the words in that one press release with their part of speech. 

C. Using the output from B, extract the adjectives and sort those adjectives from most occurrences to fewest occurrences. Print a dataframe with the 5 most frequent adjectives and their counts in the `pharma` release. See here for a list of the names of adjectives within nltk: https://pythonprogramming.net/natural-language-toolkit-nltk-part-speech-tagging/

**Resources**:

- Documentation for `.isalpha()`: https://www.w3schools.com/python/ref_string_isalpha.asp

In [14]:
pharma_alpha = [word for word in wordpunct_tokenize(pharma) 
                          if word.isalpha()]

In [15]:
pharma_pos = pos_tag(pharma_alpha)

In [16]:
adjectives = [word for word, pos in pharma_pos if pos.startswith('JJ')]

adjective_counts = {}
for adjective in adjectives:
    if adjective in adjective_counts:
        adjective_counts[adjective] += 1
    else:
        adjective_counts[adjective] = 1

sorted_adjectives = sorted(adjective_counts.items(), key=lambda x: x[1], reverse=True)

top5_df = pd.DataFrame(sorted_adjectives[:5], columns=['Adjective', 'Count'])
top5_df

,Adjective,Count
0,former,8
1,opioid,5
2,nationwide,4
3,other,3
4,addictive,3


## 1.2 named entity recognition (4 points)

A. Using the original `pharma` press release (so the one before stripping punctuation/digits), use spaCy to extract all named entities from the press release.

B. Print the unique named entities with the tag: `LAW`

In [17]:
spacy_pharma = nlp(pharma)

for one_tok in spacy_pharma.ents:
    print("Entity: " + one_tok.text + "; NER tag: " + one_tok.label_)

Entity: Insys Therapeutics Inc.; NER tag: ORG
Entity: today; NER tag: DATE
Entity: Fentanyl; NER tag: PERSON
Entity: More than 20,000; NER tag: CARDINAL
Entity: Americans; NER tag: NORP
Entity: last year; NER tag: DATE
Entity: millions; NER tag: CARDINAL
Entity: Jeff Sessions; NER tag: PERSON
Entity: This Justice Department; NER tag: ORG
Entity: Trump; NER tag: PERSON
Entity: American; NER tag: NORP
Entity: ”John N. Kapoor; NER tag: PERSON
Entity: 74; NER tag: DATE
Entity: Phoenix; NER tag: GPE
Entity: Ariz.; NER tag: GPE
Entity: the Board of Directors; NER tag: ORG
Entity: Insys; NER tag: ORG
Entity: this morning; NER tag: TIME
Entity: Arizona; NER tag: GPE
Entity: RICO; NER tag: LAW
Entity: Kapoor; NER tag: PERSON
Entity: Executive; NER tag: ORG
Entity: Board; NER tag: ORG
Entity: Insys; NER tag: ORG
Entity: Phoenix; NER tag: GPE
Entity: today; NER tag: DATE
Entity: U.S.; NER tag: GPE
Entity: District Court; NER tag: ORG
Entity: Boston; NER tag: GPE
Entity: a later date; NER tag: DAT

In [18]:
unique_law = set(entity.text for entity in spacy_pharma.ents if entity.label_ == "LAW")
for entity in unique_law:
    print(entity)

RICO
the Controlled Substances Act


C. Use Google to summarize in one sentence what the `RICO` named entity means and why this might apply to a pharmaceutical kickbacks case (and not just a mafia case...) 

RICO is used to combat organized crime, but it can also be applied to pharmaceutical kickbacks cases because it can target any enterprise engaged in ongoing criminal activity,  including schemes involving illegal payments or fraud within the pharmaceutical industry.

D. You want to extract the possible sentence lengths the CEO is facing; pull out the named entities with (1) the label `DATE` and (2) that contain the word year or years (hint: you may want to use the `re` module for that second part). Print these named entities.

In [19]:
sentence_lengths = []
for entity in spacy_pharma.ents:
    if entity.label_ == "DATE" and re.search(r'\b(?:year|years)\b', entity.text, re.IGNORECASE):
        sentence_lengths.append(entity.text)

# Print
for word in sentence_lengths:
    print(word)

last year
20 years
three years
five years
three years


E. Pull and print the original parts of the press releases where those year lengths are mentioned (e.g., the sentences or rough region of the press release). Describe in your own words (1 sentence) what length of sentence (prison) and probation (supervised release) the CEO may be facing if convicted after this indictment (if there are multiple lengths mentioned describe the maximum). 

**Hint**: you may want to use re.search or re.findall 

- For part E, you can use `re.search` and `re.findall`, or anything that works 😳.

In [20]:
# split pharma into sentences
sentences = re.split(r'(?<=[.!?]) +', pharma)

for length in sentence_lengths:
    for sentence in sentences:
        if re.search(re.escape(length), sentence):
            print(sentence)
            print("-"*50)

The founder and majority owner of Insys Therapeutics Inc., was arrested today and charged with leading a nationwide conspiracy to profit by using bribes and fraud to cause the illegal distribution of a Fentanyl spray intended for cancer patients experiencing breakthrough pain. "More than 20,000 Americans died of synthetic opioid overdoses last year, and millions are addicted to opioids.
--------------------------------------------------
Neves, Special Agent in Charge of the VA OIG Northeast Field Office.The charges of conspiracy to commit RICO and conspiracy to commit mail and wire fraud each provide for a sentence of no greater than 20 years in prison, three years of supervised release and a fine of $250,000, or twice the amount of pecuniary gain or loss.  The charges of conspiracy to violate the Anti-Kickback Law provide for a sentence of no greater than five years in prison, three years of supervised release and a $25,000 fine.
--------------------------------------------------
Neve

If convicted, the CEO risks facing up to 20 years in prison and 3 years of probation for the RICO charge and up to 5 years in prison and 3 years of probation for the Anti-Kickback Law violation. 

## 1.3 sentiment analysis  (10 points)

A. Subset the press releases to those labeled with one of three topics via `topics_clean`: Civil Rights, Hate Crimes, and Project Safe Childhood. We'll call this `doj_subset` going forward and it should have 717 rows.



In [21]:
topics = ["Civil Rights", "Hate Crimes", "Project Safe Childhood"]

doj_subset = doj[doj['topics_clean'].isin(topics)].reset_index()
len(doj_subset)
doj_subset.head()

717

,index,id,title,contents,date,topics_clean,components_clean
0,77,17-1235,Additional Former Correctional Officer Pleads Guilty to Beating of Handcuffed and Shackled Inmate at Angola State Prison,"A former supervisory correctional officer at Louisiana State Penitentiary in Angola, Louisiana, pleaded guilty yesterday in connection with the beating of a handcuffed and shackled inmate, in addition to conspiring to cover up their misconduct by falsifying official records and lying to internal investigators about what happened. James Savoy, 39, of Marksville, Louisiana, admitted during his plea hearing that he witnessed other officers using excessive force against the inmate and failed to intervene; that he conspired with other officers to cover up the beating by engaging in a variety of obstructive acts; and that he personally falsified official prison records to cover up the attack. Scotty Kennedy, 48, of Beebe, Arkansas, and John Sanders, 30, of Marksville, Louisiana previously pleaded guilty in November 2016, and September 2017, for their roles in the beating and cover up. “Every citizen has the right to due process and protection from unreasonable force, and correctional officers who violate these basic Constitutional rights must be held accountable for their egregious actions” said Acting Assistant Attorney General John Gore of the Civil Rights Division. “The Justice Department will continue to vigorously prosecute correctional officers who violate the public’s trust by committing crimes and to covering up violations of federal criminal law.” “Yesterday is another example of our office’s unwavering commitment to pursuing those who violate the federal criminal civil rights laws,” said Acting United States Attorney for the Middle District of Louisiana Corey Amundson. “We will continue to work closely with the Justice Department’s Civil Rights Division and the FBI to ensure that no one is above the law.” This case is being investigated by the FBI’s Baton Rouge Resident Agency and is being prosecuted by Assistant U.S. Attorney Frederick A. Menner, Jr. of the Middle District of Louisiana and Trial Attorney Christopher J. Perras of the Civil Rights Division’s Criminal Section.",2017-11-02T00:00:00-04:00,Civil Rights,"Civil Rights Division; USAO - Louisiana, Middle"
1,155,15-1522,Alabama Man Found Guilty of Aggravated Sexual Abuse of a Child,"A federal jury convicted Rick Lee Evans, 43, of Anniston, Alabama, today of aggravated sexual abuse of a child after a five-day trial, Assistant Attorney General Leslie R. Caldwell of the Justice Department’s Criminal Division and U.S. Attorney Joyce White Vance of the Northern District of Alabama announced. According to evidence introduced at trial, Evans, a former U.S. Army soldier, and his then-wife, a Department of Defense employee, were residing in Germany when they were asked to take temporary custody of a five-year-old child whose parents were deployed to Iraq with the U.S. Army. Evans sexually abused the child on multiple occasions during the 18 months that the child lived with him from May 2007 to December 2008. Trial Attorney Austin M. Berry of the Criminal Division’s Child Exploitation and Obscenity Section (CEOS) and Assistant U.S. Attorney Jacquelyn Hutzell of the Northern District of Alabama are prosecuting the case. U.S. Army Criminal Investigations Division and the FBI’s Birmingham, Alabama, Division investigated the case. This case was brought as part of Project Safe Childhood, a nationwide initiative to combat the growing epidemic of child sexual exploitation and abuse, launched in May 2006 by the Department of Justice. Led by U.S. Attorneys’ offices and CEOS, Project Safe Childhood marshals federal, state and local resources to better locate, apprehend and prosecute individuals who exploit children via the Internet, as well as to identify and rescue victims. For more information about Project Safe Childhood, please visit www.justice.gov/psc.",2015-12-11T00:00:00-05:00,Project S

B. Write a function that takes one press release string as an input and:

- Removes named entities from each press release string (**Hint**: you may want to use `re.sub` with an or condition)
- Scores the sentiment of the entire press release using the `SentimentIntensityAnalyzer` and `polarity_scores`
- Returns the length-four (negative, positive, neutral, compound) sentiment dictionary (any order is fine)

Apply that function to each of the press releases in `doj_subset`. 

**Hints**: 

- A function + list comprehension to execute will takes about 30 seconds on a respectable local machine and about 2 mins on jhub; if it's taking a very long time, you may want to check your code for inefficiencies. If you can't fix those, for partial credit on this part/full credit on remainder, you can take a small random sample of the 717


In [22]:
def processer(string):
    
    # Process the press release using spaCy
    doc = nlp(string)

    # Remove named entities
    for ent in doc.ents:
        string = re.sub(re.escape(ent.text), '', string) # escape handles entities with characters that have special meanings in regex

    # Initialize SentimentIntensityAnalyzer
    sia = SentimentIntensityAnalyzer()

    # Score the sentiment of the press release
    sentiment_scores = sia.polarity_scores(string)

    # Return the length-four sentiment dictionary
    return sentiment_scores

# Test with example string
str = "PORTLAND, Oregon. – Mohamed Osman Mohamud, 23, who was convicted in 2013 of attempting to use a weapon of mass destruction (explosives) in connection with a plot to detonate a vehicle bomb at an annual Christmas tree lighting ceremony in Portland, was sentenced today to serve 30 years in prison, followed by a lifetime term of supervised release. Mohamud, a naturalized U.S. citizen from Somalia and former resident of Corvallis, Oregon, was arrested on Nov. 26, 2010, after he attempted to detonate what he believed to be an explosives-laden van that was parked near the tree lighting ceremony in Portland.  The arrest was the culmination of a long-term undercover operation, during which Mohamud was monitored closely for months as his bomb plot developed.  The device was in fact inert, and the public was never in danger from the device. At sentencing, United States District Court Judge Garr M. King, who presided over Mohamed’s 14-day trial, said “the intended crime was horrific,” and that the defendant, even though he was presented with options by undercover FBI employees, “never once expressed a change of heart.”  King further noted that the Christmas tree ceremony was attended by up to 10,000 people, and that the defendant “wanted everyone to leave either dead or injured.”  King said his sentence was necessary in view of the seriousness of the crime and to serve as deterrence to others who might consider similar acts.     “With today’s sentencing, Mohamed Osman Mohamud is being held accountable for his attempted use of what he believed to be a massive bomb to attack innocent civilians attending a public Christmas tree lighting ceremony in Portland,” said John P. Carlin, Assistant Attorney General for National Security.  “The evidence clearly indicated that Mohamud was intent on killing as many people as possible with his attack.  Fortunately, law enforcement was able to identify him as a threat, insert themselves in the place of a terrorist that Mohamud was trying to contact, and thwart Mohamud’s efforts to conduct an attack on our soil.  This case highlights how the use of undercover operations against would-be terrorists allows us to engage and disrupt those who wish to commit horrific acts of violence against the innocent public.  The many agents, analysts, and prosecutors who have worked on this case deserve great credit for their roles in protecting Portland from the threat posed by this defendant and ensuring that he was brought to justice.” “This trial provided a rare glimpse into the techniques Al Qaeda employs to radicalize home-grown extremists,” said Amanda Marshall, U.S. Attorney for the District of Oregon.  “With the sentencing today, the court has held this defendant accountable.   I thank the dedicated professionals in the law enforcement and intelligence communities who were responsible for this successful outcome.  I look forward to our continued work with Muslim communities in Oregon who are committed to ensuring that all young people are safe from extremists who seek to radicalize others to engage in violence.”  According to the trial evidence, in February 2009, Mohamud began communicating via e-mail with Samir Khan, a now-deceased al Qaeda terrorist who published Jihad Recollections, an online magazine that advocated violent jihad, and who also published Inspire, the official magazine of al-Qaeda in the Arabian Peninsula.  Between February and August 2009, Mohamed exchanged approximately 150 emails with Khan.  Mohamud wrote several articles for Jihad Recollections that were published under assumed names. In August 2009, Mohamud was in email contact with Amro Al-Ali, a Saudi national who was in Yemen at the time and is today in custody in Saudi Arabia for terrorism offenses.  Al-Ali sent Mohamud detailed e-mails designed to facilitate Mohamud’s travel to Yemen to train for violent jihad.  In December 2009, while Al-Ali was in the northwest frontier province of Pakistan, Mohamud and Al-Ali discussed the possibility of Mohamud traveling to Pakistan to join Al-Ali in terrorist activities. Mohamud responded to Al-Ali in an e-mail: “yes, that would be wonderful, just tell me what I need to do.”  Al-Ali referred Mohamud to a second associate overseas and provided Mohamud with a name and email address to facilitate the process. In the following months, Mohamud made several unsuccessful attempts to contact Al-Ali’s associate.  Ultimately, an FBI undercover operative contacted Mohamud via email under the guise of being an associate of Al-Ali’s.  Mohamud and the FBI undercover operative agreed to meet in Portland in July 2010.  At the meeting, Mohamud told the FBI undercover operative he had written articles that were published in Jihad Recollections.  Mohamud also said that he wanted to become “operational.”  Asked what he meant by “operational,” Mohamud said he wanted to put an explosion together, but needed help. According to evidence presented at trial, at a meeting in August 2010, Mohamud told undercover FBI operatives he had been thinking of committing violent jihad since the age of 15.  Mohamud then told the undercover FBI operatives that he had identified a potential target for a bomb: the annual Christmas tree lighting ceremony in Portland’s Pioneer Courthouse Square on Nov. 26, 2010.  The undercover FBI operatives cautioned Mohamud several times about the seriousness of this plan, noting there would be many people at the event, including children, and emphasized that Mohamud could abandon his attack plans at any time with no shame.  Mohamud indicated the deaths would be justified and that he would not mind carrying out a suicide attack on the crowd. According to evidence presented at trial, in the ensuing months Mohamud continued to express his interest in carrying out the attack and worked on logistics.  On Nov. 4, 2010, Mohamud and the undercover FBI operatives traveled to a remote location in Lincoln County, Oregon, where they detonated a bomb concealed in a backpack as a trial run for the upcoming attack.  During the drive back to Corvallis, Mohamud was asked if was capable looking at all the bodies of those who would be killed during the explosion.  In response, Mohamud noted, “I want whoever is attending that event to be, to "

processer(str)

{'neg': 0.122, 'neu': 0.815, 'pos': 0.063, 'compound': -0.9958}

In [45]:
sentiments = doj_subset['contents'].apply(processer).apply(pd.Series)
sentiments

,neg,neu,pos,compound
0,0.200,0.751,0.049,-0.9931
1,0.129,0.804,0.066,-0.9325
2,0.090,0.835,0.075,-0.7579
3,0.121,0.798,0.081,-0.9037
4,0.175,0.782,0.043,-0.9864
...,...,...,...,...
712,0.159,0.778,0.062,-0.9737
713,0.081,0.825,0.095,0.5267
714,0.150,0.758,0.092,-0.9571
715,0.133,0.745,0.122,-0.8783


C. Add the four sentiment scores to the `doj_subset` dataframe to create a dataframe: `doj_subset_wscore`. Sort from highest neg to lowest neg score and print the top `id`, `contents`, and `neg` columns of the two most neg press releases. 

Notes:

- Don't worry if your sentiment score differs slightly from our output on GitHub; differences in preprocessing can lead to diff scores

In [24]:
# Merge the dataframes based on their indices
doj_subset_wscores = pd.concat([doj_subset.reset_index(drop=True), sentiments], axis=1)
doj_wscores_sorted = doj_subset_wscores.sort_values(by='neg', ascending=False)
doj_wscores_sorted[['id','contents','neg']].head(2)

,id,contents,neg
13,14-248,"The Department of Justice announced that this morning John W. Ng, 58, of Albuquerque, N.M., made his initial appearance in federal court on a criminal complaint charging him with a hate crime offense. This charge is related to anti-Semitic threats Ng made against a Jewish woman who owns and operates the Nosh Jewish Delicatessen and Bakery in Albuquerque. Ng was arrested by the FBI on March 7, 2014, based on a criminal complaint alleging that he interfered with the victim’s federally protected rights by threatening her and interfering with her business because of her religion. According to the criminal complaint, between Jan. 22, 2014, and Feb. 8, 2014, Ng allegedly posted threatening anti-Semitic notes on and in the vicinity of the victim’s business. A criminal complaint merely establishes probable cause, and Ng is presumed innocent unless proven guilty. If convicted on the offense charged in the criminal complaint, Ng faces a maximum statutory penalty of one year in prison. This matter was investigated by the Albuquerque Division of the FBI and is being prosecuted by Assistant U.S. Attorney Mark T. Baker of the U.S. Attorney’s Office for the District of New Mexico and Trial Attorney AeJean Cha of the U.S. Department of Justice’s Civil Rights Division.",0.319
34,13-312,"John Hall, 27, an Aryan Brotherhood member and inmate at the Federal Correctional Institution (FCI) in Seagoville, Texas, was sentenced today by U.S. District Judge Reed O’Connor after pleading guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act stemming from his assault of a fellow inmate, whom he believed to be gay, the Department of Justice announced. Hall assaulted his fellow inmate with a dangerous weapon, causing bodily injury to the victim on Dec. 20, 2011. Hall was sentenced to serve 71 months in prison to be served consecutively with the sentence he is currently serving. The assault occurred on Dec. 20, 2011, inside the FCI Seagoville when Hall targeted and attacked the victim, a fellow inmate, because he believed the victim was gay or involved in a sexual relationship with another male inmate. Hall repeatedly punched, kicked and stomped on the victim’s face with his shod feet, a dangerous weapon, while yelling a homophobic slur. The victim lost consciousness during the assault and suffered multiple lacerations to his face. The victim also sustained a fractured eye socket, lost a tooth, fractured other teeth and was treated at a hospital for the injuries he sustained during Hall’s unprovoked attack. Hall pleaded guilty to violating the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act on Nov. 8, 2012. “Brutality and violence based on sexual orientation has no place in a civilized society,” said Thomas E. Perez, Assistant Attorney General for the Civil Rights Division. “The Justice Department is committed to using all the tools in our law enforcement arsenal, including the Matthew Shepard and James Byrd Jr. Hate Crimes Prevention Act, to prosecute acts motivated by hate.” “This prosecution sends a clear message that this office, in partnership with attorneys in the department’s Civil Rights Division, will prioritize and aggressively prosecute hate crimes and others civil rights violations in North Texas,” said U.S. Attorney Sarah R. Saldaña of the Northern District of Texas. This case was investigated by the FBI Dallas Division. The case was prosecuted by Assistant U.S. Attorney Errin Martin and Trial Attorney Adriana Vieco of the Civil Rights Division.",0.300


D. With the dataframe from part C, find the mean compound sentiment score for each of the three topics in `topics_clean` using group_by and agg.

E. Add a 1 sentence interpretation of why we might see the variation in scores (remember that compound is a standardized summary where -1 is most negative; +1 is most positive)


In [25]:
mean_scores = doj_subset_wscores.groupby('topics_clean').agg({'compound': 'mean'}).reset_index()
mean_scores

,topics_clean,compound
0,Civil Rights,-0.087140
1,Hate Crimes,-0.935576
2,Project Safe Childhood,-0.653711


Press releases related to hate crimes are scored negatively because they likely contain distressing information while those related to civil rights are likely announcing progress or efforts by the DOJ to improve civil rights, explaining why they are more positive.

# 2. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 2.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [26]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                       "trial", "assistance", "assist"]

In [27]:
def preprocess_text(raw_string):
    # Initialize stemmer
    stemmer = SnowballStemmer("english")
    
    # Convert to lowercase
    tokens = raw_string.lower().split()
    
    # Remove stopwords (both general and custom) and only retain alpha words that are at least 4 characters long
    all_stopwords = stopwords.words('english') + custom_doj_stopwords
    tokens = [word for word in tokens if word not in all_stopwords and word.isalpha() and len(word) >= 4]
    
    # Stem the words
    tokens = [stemmer.stem(token) for token in tokens]
    
    return ' '.join(tokens)

In [28]:
doj_subset_wscores['processed_text'] = doj_subset_wscores['contents'].apply(preprocess_text)

In [29]:
example_rows = doj_subset_wscores[doj_subset_wscores['id'].isin(['16-718', '16-217'])]
example_rows[['id', 'contents', 'processed_text']]

,id,contents,processed_text
313,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-involved shootings and to more effectively and quickly investigate officer-involved shootings that do occur, through measures that include: “This settlement represents a renewed commitment by the city of Miami and Chief Rodolfo Llanes to provide constitutional policing for Miami residents and to protect public safety through sustainable reform,” said Principal Deputy Assistant Attorney General Gupta. “The agreement will help to strengthen the relationship between the MPD and the communities they serve by improving accountability for officers who fire their weapons unlawfully, and provides for community participation in the enforcement of this agreement.” “Today's agreement is the result of a joint effort between the Department of Justice and the City of Miami to ensure that the Miami Police Department continues its efforts to make our community safe while protecting the sacred Constitutional rights of all of our citizens,” said U.S. Attorney Ferrer. “Through oversight and communication, the agreement seeks to make permanent the positive changes that former Chief Orosa and Chief Llanes have made, and we applaud the City Commission’s vote.” The settlement agreement builds upon important reforms implemented by the city since the Justice Department issued its findings, including: The investigation was conducted by attorneys and staff from the Civil Rights Division’s Special Litigation Section and the Civil Division of the U. S. Attorney’s Office of the Southern District of Florida.",reach comprehens settlement agreement citi miami miami polic resolv shoot announc princip deputi general vanita head wifredo ferrer southern approv citi commiss today effect agreement sign resolv claim stem shoot conduct violent crime control enforc issu juli identifi pattern practic excess forc shoot violat fourth amend complianc settlement monitor independ former polic chief jane settlement citi implement comprehens reform ensur constitut polic support public settlement agreement design minim shoot effect quick investig shoot measur settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain said princip deputi general agreement help strengthen relationship communiti serv improv account offic fire weapon provid communiti particip enforc agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr constitut said oversight agreement seek make perman posit chang former chief orosa chief llane applaud citi settlement agreement build upon import reform implement citi sinc issu conduct attorney staff special litig section southern


## 2.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [30]:
def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names_out())
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [31]:
# Part A
metadata = doj_subset_wscores[['id', 'compound', 'topics_clean', 'processed_text']]

string_list = doj_subset_wscores['processed_text']

dtm_df = create_dtm(string_list, metadata)

In [32]:
# Part B

# Function to extract top words for a subset of data
def get_topwords(subset, text, top_n):
    # Create a CountVectorizer
    vectorizer = CountVectorizer(lowercase=True)

    # Fit vectorizer to text data in the subset
    dtm = vectorizer.fit_transform(subset[text])

    # Get word counts
    word_counts = dtm.sum(axis=0)

    # Create df with term frequencies
    term_frequencies = pd.DataFrame({'term': vectorizer.get_feature_names_out(), 'frequency': word_counts.flat})

    # Sort by frequency in descending order and extract top words
    top_words = term_frequencies.sort_values(by='frequency', ascending=False).head(top_n)

    return top_words

# Get top 5% threshold
top_threshold = metadata['compound'].quantile(0.95)

# Filter to press releases meeting this threshold and get top 10 words
top5_releases = metadata[metadata['compound'] >= top_threshold]
get_topwords(top5_releases, 'processed_text', top_n=10)


,term,frequency
45,agreement,138
490,enforc,118
499,ensur,110
1415,state,95
1320,said,78
1362,settlement,75
627,general,74
565,feder,72
262,communiti,72
1655,work,71


In [33]:
# Part C

# Get bottom 5% threshold
bottom_threshold = metadata['compound'].quantile(0.05)

# Filter to press releases meeting this threshold and get top 10 words
bottom5_releases = metadata[metadata['compound'] <= bottom_threshold]
get_topwords(bottom5_releases, 'processed_text', top_n=10)

,term,frequency
376,crime,130
1098,offic,121
113,assault,116
721,hate,110
408,defend,109
597,feder,90
1410,sentenc,88
1246,prosecut,87
1695,victim,84
705,guilti,83


In [34]:
# Part D
topics = ["Civil Rights", "Hate Crimes", "Project Safe Childhood"]

for topic in topics:
    # Filter to specific topic
    filtered = metadata[metadata['topics_clean'] == topic]
    
    # Call the get_topwords function to get the top words for this topic
    top_words = get_topwords(filtered,'processed_text', 10)
    
    # Print the top words for the current topic
    print(f"Top 10 Words for '{topic}':")
    for i, row in top_words.iterrows():
        print(f"{i + 1}. {row['term']} ({row['frequency']} times)")
    print()

Top 10 Words for 'Civil Rights':
1746. hous (588 times)
2546. offic (504 times)
1218. enforc (498 times)
3210. said (490 times)
1043. discrimin (476 times)
1373. feder (459 times)
3955. violat (444 times)
113. alleg (389 times)
1521. general (386 times)
3472. state (347 times)

Top 10 Words for 'Hate Crimes':
2313. prosecut (458 times)
703. crime (445 times)
1343. hate (421 times)
2597. sentenc (420 times)
1088. feder (418 times)
494. charg (406 times)
2528. said (404 times)
767. defend (398 times)
1298. guilti (374 times)
3216. year (306 times)

Top 10 Words for 'Project Safe Childhood':
370. child (995 times)
768. exploit (673 times)
1966. sexual (570 times)
1898. safe (475 times)
1743. project (472 times)
512. crimin (404 times)
1750. prosecut (357 times)
1952. sentenc (329 times)
372. children (313 times)
1114. investig (256 times)



## 2.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [35]:
from gensim.models import LdaModel
import gensim

# Convert preprocessed text data to list of documents
documents = [text.split() for text in doj_subset_wscores['processed_text']]

# Create a dictionary and corpus
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]

# Estimate the LDA model with 3 topics
lda_model = LdaModel(corpus, num_topics=3, id2word=dictionary, passes=10)

# Print the topics and top words
topics = lda_model.print_topics(num_words=10)
for topic in topics:
    print(topic)

(0, '0.010*"enforc" + 0.009*"discrimin" + 0.008*"offic" + 0.008*"feder" + 0.007*"said" + 0.007*"employ" + 0.006*"ensur" + 0.006*"polic" + 0.006*"requir" + 0.006*"general"')
(1, '0.022*"hous" + 0.013*"disabl" + 0.010*"fair" + 0.009*"individu" + 0.009*"discrimin" + 0.008*"access" + 0.008*"said" + 0.007*"alleg" + 0.007*"violat" + 0.006*"state"')
(2, '0.012*"child" + 0.012*"prosecut" + 0.011*"sentenc" + 0.009*"charg" + 0.009*"guilti" + 0.008*"investig" + 0.008*"exploit" + 0.008*"feder" + 0.008*"sexual" + 0.008*"crimin"')


## 2.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [36]:
## get doc-level topic probabilities 
doc_topics = [lda_model.get_document_topics(item, minimum_probability=0) for item in corpus]

# Make sure length matches our dataframe's number of rows
assert len(doc_topics) == len(doj_subset_wscores)

In [37]:
## add those topic probabilities to the dataframe
topic_df = pd.DataFrame([[tup[1] for tup in topic] for topic in doc_topics], 
                        columns=[f'topic_{i+1}' for i in range(3)])

# Concatenate topic probabilities to the main dataframe
doj_subset_wscores = pd.concat([doj_subset_wscores, topic_df], axis=1)

# Determine the top topic for each document
doj_subset_wscores['top_topic'] = doj_subset_wscores[['topic_1', 'topic_2', 'topic_3']].idxmax(axis=1)

In [38]:
## summarize the topic proportions for each of the topics_clean 
ct = pd.crosstab(doj_subset_wscores['topics_clean'], doj_subset_wscores['top_topic'], normalize='index') * 100
print(ct)

top_topic                 topic_1    topic_2    topic_3
topics_clean                                           
Civil Rights            39.344262  28.196721  32.459016
Hate Crimes              0.406504   0.406504  99.186992
Project Safe Childhood   0.602410   0.000000  99.397590


# 3. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [39]:
# Part A
def create_bigram_onedoc(text):
    # Split text into words
    words = text.split()
    
    # Create bigrams using zip and list comprehension
    bigrams = ["_".join(bigram) for bigram in zip(words[:-1], words[1:])]
    
    # Join bigrams into single string
    return " ".join(bigrams)

# Apply the function to the processed_text column to create processed_text_bigrams column
doj_subset_wscores['processed_text_bigrams'] = doj_subset_wscores['processed_text'].apply(create_bigram_onedoc)

In [40]:
# Part B
doj_subset_wscores[doj_subset_wscores['id']=="16-217"][['id', 'processed_text', 'processed_text_bigrams']]

,id,processed_text,processed_text_bigrams
313,16-217,reach comprehens settlement agreement citi miami miami polic resolv shoot announc princip deputi general vanita head wifredo ferrer southern approv citi commiss today effect agreement sign resolv claim stem shoot conduct violent crime control enforc issu juli identifi pattern practic excess forc shoot violat fourth amend complianc settlement monitor independ former polic chief jane settlement citi implement comprehens reform ensur constitut polic support public settlement agreement design minim shoot effect quick investig shoot measur settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain said princip deputi general agreement help strengthen relationship communiti serv improv account offic fire weapon provid communiti particip enforc agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr constitut said oversight agreement seek make perman posit chang former chief orosa chief llane applaud citi settlement agreement build upon import reform implement citi sinc issu conduct attorney staff special litig section southern,reach_comprehens comprehens_settlement settlement_agreement agreement_citi citi_miami miami_miami miami_polic polic_resolv resolv_shoot shoot_announc announc_princip princip_deputi deputi_general general_vanita vanita_head head_wifredo wifredo_ferrer ferrer_southern southern_approv approv_citi citi_commiss commiss_today today_effect effect_agreement agreement_sign sign_resolv resolv_claim claim_stem stem_shoot shoot_conduct conduct_violent violent_crime crime_control control_enforc enforc_issu issu_juli juli_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_shoot shoot_violat violat_fourth fourth_amend amend_complianc complianc_settlement settlement_monitor monitor_independ independ_former former_polic polic_chief chief_jane jane_settlement settlement_citi citi_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_settlement settlement_agreement agreement_design design_minim minim_shoot shoot_effect effect_quick quick_investig investig_shoot shoot_measur measur_settlement settlement_repres repres_renew renew_commit commit_citi citi_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_sustain sustain_said said_princip princip_deputi deputi_general general_agreement agreement_help help_strengthen strengthen_relationship relationship_communiti communiti_serv serv_improv improv_account account_offic offic_fire fire_weapon weapon_provid provid_communiti communiti_particip particip_enforc enforc_agreement agreement_result result_joint joint_effort effort_citi citi_miami miami_ensur ensur_miami miami_polic polic_continu continu_effort effort_make make_communiti communiti_safe safe_protect protect_sacr sacr_constitut constitut_said said_oversight oversight_agreement agreement_seek seek_make make_perman perman_posit posit_chang chang_former former_chief chief_orosa orosa_chief chief_llane llane_applaud applaud_citi citi_settlement settlement_agreement agreement_build build_upon upon_import import_reform reform_implement implement_citi citi_sinc sinc_issu issu_conduct conduct_attorney attorney_staff staff_special special_litig litig_section section_southern


C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [50]:
# Part C
metadata = doj_subset_wscores[['id', 'compound', 'topics_clean', 'processed_text_bigrams']]

string_list = doj_subset_wscores['processed_text_bigrams']

dtm_bigram = create_dtm(string_list, metadata)

In [48]:
# Part D
dtm_df.shape
dtm_bigram.shape

(717, 6112)

(717, 63646)

In [43]:
topics = ["Civil Rights", "Hate Crimes", "Project Safe Childhood"]

for topic in topics:
    # Filter to specific topic
    filtered = metadata[metadata['topics_clean'] == topic]
    
    # Call the get_topwords function to get the top words for this topic
    top_words = get_topwords(filtered,'processed_text_bigrams', 10)
    
    # Print the top words for the current topic
    print(f"Top 10 Words for '{topic}':")
    for i, row in top_words.iterrows():
        print(f"{i + 1}. {row['term']} ({row['frequency']} times)")
    print()

Top 10 Words for 'Civil Rights':
12437. fair_hous (225 times)
9120. deputi_general (221 times)
24787. princip_deputi (221 times)
33860. vanita_head (200 times)
14168. general_vanita (199 times)
28688. said_princip (186 times)
2052. announc_today (116 times)
33450. unit_state (106 times)
6937. consent_decre (92 times)
490. act_general (80 times)

Top 10 Words for 'Hate Crimes':
8249. hate_crime (299 times)
13718. plead_guilti (269 times)
16822. special_agent (117 times)
19970. year_prison (104 times)
17774. thoma_general (95 times)
7841. grand_juri (93 times)
15794. said_thoma (91 times)
179. act_general (85 times)
6582. face_maximum (77 times)
6790. feder_hate (76 times)

Top 10 Words for 'Project Safe Childhood':
8578. project_safe (472 times)
1781. child_exploit (266 times)
1832. child_pornographi (237 times)
9836. sexual_exploit (218 times)
8035. plead_guilti (188 times)
1855. child_sexual (177 times)
3881. exploit_obscen (175 times)
7359. obscen_section (161 times)
9310. safe_child

# 4. Optional extra credit (2 points)

You notice that the pharmaceutical kickbacks press release we analyzed in question 1 was for an indictment, and that in the original data, there's not a clear label for whether a press release outlines an indictment (charging someone with a crime), a conviction (convicting them after that charge either via a settlement or trial), or a sentencing (how many years of prison or supervised release a defendant is sentenced to after their conviction).

You want to see if you can identify pairs of press releases where one press release is from one stage (e.g., indictment) and another is from a different stage (e.g., a sentencing).

You decide that one way to approach is to find the pairwise string similarity between each of the processed press releases in `doj_subset`. There are many ways to do this, so Google for some approaches, focusing on ones that work well for entire documents rather than small strings.

Find the top two pairs (so four press releases total)-- do they seem like different stages of the same crime or just press releases covering similar crimes?

In [44]:
# your code here 